In [3]:
import csv
import random
import math

def loadcsv(filename):
    lines = csv.reader(open(filename,"r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def seperateByClass(dataset):
    seperated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if(vector[-1] not in seperated):
            seperated[vector[-1]] = []
        seperated[vector[-1]].append(vector)
    return seperated

def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers]) / float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    seperated = seperateByClass(dataset)
    summaries = {}
    for classValue, instances in seperated.items():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def main():
    filename = 'dataset.csv'
    splitRatio = 0.67
    dataset = loadcsv(filename)
    print("\n The length of the Data Set : ",len(dataset))
    print("\n The Data Set Splitting into Training and Testing \n")
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('\n Number of Rows in Training Set:{0} rows'.format(len(trainingSet)))
    print('\n Number of Rows in Testing Set:{0} rows'.format(len(testSet)))
    print("\n First Five Rows of Training Set:\n")
    for i in range(0,5):
        print(trainingSet[i],"\n")

    print("\nFirst Five Rows of Testing Set:\n")
    for i in range(0,5):
        print(testSet[i],"\n")

    #prepare model
    summaries = summarizeByClass(trainingSet)
    print("\n Model Summaries:\n",summaries)

    predictions = getPredictions(summaries, testSet)
    print("\nPredictions:\n",predictions)

    accuracy = getAccuracy(testSet, predictions)
    print('\n Accuracy : {0}%'.format(accuracy))
main()


 The length of the Data Set :  768

 The Data Set Splitting into Training and Testing 


 Number of Rows in Training Set:514 rows

 Number of Rows in Testing Set:254 rows

 First Five Rows of Training Set:

[4.0, 154.0, 72.0, 29.0, 126.0, 31.3, 0.338, 37.0, 0.0] 

[6.0, 117.0, 96.0, 0.0, 0.0, 28.7, 0.157, 30.0, 0.0] 

[7.0, 109.0, 80.0, 31.0, 0.0, 35.9, 1.127, 43.0, 1.0] 

[7.0, 136.0, 90.0, 0.0, 0.0, 29.9, 0.21, 50.0, 0.0] 

[2.0, 100.0, 54.0, 28.0, 105.0, 37.8, 0.498, 24.0, 0.0] 


First Five Rows of Testing Set:

[1.0, 103.0, 30.0, 38.0, 83.0, 43.3, 0.183, 33.0, 0.0] 

[1.0, 115.0, 70.0, 30.0, 96.0, 34.6, 0.529, 32.0, 1.0] 

[8.0, 99.0, 84.0, 0.0, 0.0, 35.4, 0.388, 50.0, 0.0] 

[10.0, 125.0, 70.0, 26.0, 115.0, 31.1, 0.205, 41.0, 1.0] 

[13.0, 145.0, 82.0, 19.0, 110.0, 22.2, 0.245, 57.0, 0.0] 


 Model Summaries:
 {0.0: [(3.3970588235294117, 3.025650937459514), (109.81176470588235, 26.020524740886387), (67.74705882352941, 18.441508191097373), (19.220588235294116, 14.967856994766828)